## Supply Planning using Gurobi
This notebook demostrates how supply planning optmization can be done using Gurobi using a simplied Supply Planning Problem usecase.

Supply planning is managing the inventory produced by manufacturing to fulfil the requirements created from the demand plan.

### How many pallets do we need to ship to DC1 in the near future?

![suppy_planning_problem.png](img/suppy_planning_problem.png)

_source_: [Samir Saci](https://towardsdatascience.com/supply-planning-using-linear-programming-with-python-bff2401bf270)


### Problem Statement

As a Supply Planning manager of a mid-size manufacturing company, you received the feedback that the distribution costs are too high. Based on the analysis of the Transportation Manager this is mainly due to the stock allocation rules.

In some cases, your customers are not shipped by the closest distribution centre, which impacts your freight costs.

For simplicity, let's say we have the following points to take into consideration:

- Inbound Transportation Costs from the Plants to the Distribution Centers (DC) ($/Carton).

- Outbound Transportation Costs from the DCs to the final customer ($/Carton).

- Customer Demand (Carton).

This problem statement is based on [this](https://towardsdatascience.com/supply-planning-using-linear-programming-with-python-bff2401bf270) article. There the problem was solved using PuLP framework. 

### Loading data

- Loading the near future demand of stores (or customers) in terms of number of pallets required into a dataframe.

In [93]:
'''
Author: Dhruva Ahuja
'''
import pandas as pd

df_demand = pd.read_csv('data/df_demand.csv', index_col=0)
df_demand.set_index('STORE', inplace=True)
df_demand.head()

,DEMAND
STORE,
S1,244
S2,172
S3,124
S4,90
S5,158


### Loading data
- Inbound transportation cost from plant $P_i$ to distribution center $DC_j$ loaded into `df_inbound`.
- Outbound transportation cost from distribution center $DC_j$ to supplier $S_k$ loaded into `df_outbound`.

In [94]:
df_inbound = pd.read_csv('data/df_inbound_price.csv', index_col=0)
df_outbound = pd.read_csv('data/df_outbound_price.csv', index_col=0)
df_outbound = df_outbound.set_index('from')
df_outbound.head()

,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,...,S191,S192,S193,S194,S195,S196,S197,S198,S199,S200
from,,,,,,,,,,,,,,,,,,,,,
D1,2.30,4.23,2.26,3.38,1.59,2.01,5.32,6.63,2.38,6.62,...,5.86,8.30,3.02,1.01,2.77,2.96,3.53,8.60,2.77,7.06
D2,5.31,2.18,8.52,8.34,4.59,1.04,1.89,6.45,8.35,3.32,...,7.54,2.11,4.33,1.54,4.75,7.84,8.21,4.51,3.27,3.13


In [95]:
df_inbound

,D1,D2
P1,3.0,5.0
P2,2.3,6.6


In [96]:
pq = df_inbound.unstack()
pq

D1  P1    3.0
    P2    2.3
D2  P1    5.0
    P2    6.6
dtype: float64

In [97]:
pq.groupby(level=0).sum()

D1     5.3
D2    11.6
dtype: float64

In [98]:
unstack_outbound = df_outbound.unstack()
unstack_inbound = df_inbound.unstack()

The optimization problem can be formulated as follows:

$$ TC = \sum_{i=1}^{2} \sum_{j=1}^{2} IB_{i, j} \times I_{i, j} + \sum_{j=1}^{2} \sum_{k=1}^{200} OB_{j, k} \times O_{j, k} $$

Where, 
- $IB_{i, j}$ is inbound cost (\$/pallete) from plant $P_i$ to $DC_j$.
- $OB_{j, k}$ is outbound cost (\$/pallete) from $DC_j$ to store $S_k$.


We have to minimize TC or Total Cost, subjected to some contraints (given later).

$$ \text{minimize} \quad {TC}$$

In [99]:
import gurobipy as gb
import gurobipy_pandas as gbpd
from gurobipy import GRB

m = gb.Model('supply_planning.lp')

# Define inbound and outbound variables, since the number of palletes (inbound or outbound) can only be an integer, 
# we specify the vtype to be integer. Hence making it a mixed integer linear programming problem.
I = gbpd.add_vars(m, unstack_inbound, vtype=GRB.INTEGER, name='inbound')
O = gbpd.add_vars(m, unstack_outbound, vtype=GRB.INTEGER, name='outbound')

m.update()

In [100]:
unstack_inbound

D1  P1    3.0
    P2    2.3
D2  P1    5.0
    P2    6.6
dtype: float64

In [101]:
objective = gb.quicksum(I * unstack_inbound)
objective += gb.quicksum(O * unstack_outbound)

m.setObjective(objective, GRB.MINIMIZE)
m.update()

Equality Constraint: Total inbound should be equal to total outbound for a distribution center.


$$ \sum_{i=0}^{2} I_{i, j} = \sum_{k=0}^{200} O_{j, k} ~~~~~~~~ \forall ~~ j \in \{1, 2\}$$

Inequality Contraint: Outbound from distribution centers should meet the customer demand.

$$ \sum_{j=0}^{2} O_{j, k} \ge D_k ~~~~~~~~ \forall ~~ k \in \{1, 2, \dotsc, 200\}$$

Integrality contraint: 
$$I_{i, j} \in \mathbb{Z}^+ ~~~ \forall (i, j), \\ O_{j, k} \in \mathbb{Z}^+ ~~~ \forall (j, k)$$

In [102]:
# Add the equality contraints
gbpd.add_constrs(m, I.groupby(level=0).agg(gb.quicksum), GRB.EQUAL, O.groupby(level=1).agg(gb.quicksum))

# Add the inquality contraints
gbpd.add_constrs(m, O.groupby(level=0).agg(gb.quicksum), GRB.GREATER_EQUAL, df_demand['DEMAND'])

m.update()

In [103]:
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 202 rows, 404 columns and 804 nonzeros
Model fingerprint: 0xb607bbb8
Variable types: 0 continuous, 404 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 3e+02]
Found heuristic solution: objective 285354.69000
Presolve removed 202 rows and 404 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 217189 285355 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.171893200000e+05, best bound 2.171893200000e+05, gap 0.0000%


In [104]:
I.reset_index().rename(columns={'level_0': 'to', 'level_1': 'from'}) \
.pivot(index='from', columns='to', values='inbound').map(lambda x: int(x.X))

to,D1,D2
from,,
P1,0,6232
P2,25574,0


In [105]:
O.reset_index().rename(columns={'level_0': 'to'}).pivot(index='from', columns='to', values='outbound')\
    .map(lambda x: int(x.X))

to,S1,S10,S100,S101,S102,S103,S104,S105,S106,S107,...,S90,S91,S92,S93,S94,S95,S96,S97,S98,S99
from,,,,,,,,,,,,,,,,,,,,,
D1,244,0,214,270,0,121,0,106,0,225,...,0,118,193,59,67,0,0,96,148,98
D2,0,129,0,0,110,0,257,0,175,0,...,249,0,0,0,0,218,187,0,0,0


In [107]:
df_demand.T

STORE,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,...,S191,S192,S193,S194,S195,S196,S197,S198,S199,S200
DEMAND,244,172,124,90,158,175,269,223,123,129,...,250,39,99,178,47,57,52,243,70,50
